# Task 4
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline, get_scheduler, AutoTokenizer, DistilBertModel

from torch.optim import AdamW

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model_transformer = DistilBertModel.from_pretrained("distilbert-base-uncased")
#nlp = pipeline("sentiment-analysis", model="model/distilbert-base-uncased-finetuned-sst-2-english")
# Add any other imports you need here

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Depending on your approach, you might need to adapt the structure of this template or parts not marked by TODOs.
It is not necessary to completely follow this template. Feel free to add more code and delete any parts that are not required.

In [12]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)
BATCH_SIZE =4  # TODO: Set the batch size according to both training performance and available memory
NUM_EPOCHS = 15  # TODO: Set the number of epochs

train_val = pd.read_csv("train.csv")
test_val = pd.read_csv("test_no_score.csv")
#len(test_val)
#test_val["title"]+". "+test_val["sentence"]
#test_val["title"][0]
index=55
cat=train_val.iloc[index]["title"]+". "+train_val.iloc[index]["sentence"]
inputs = tokenizer(cat, return_tensors="pt")
#result=nlp(cat)
#print(result)
outputembedding = model_transformer(**inputs)
print(outputembedding.last_hidden_state.squeeze().size())
#print(outputembedding)
model_transformer=model_transformer.to(DEVICE)

cpu
torch.Size([54, 768])


In [13]:
# TODO: Fill out ReviewDataset 
class ReviewDataset(Dataset):
    def __init__(self, data_frame,train=True):
        self.data=data_frame
        self.train=train
        pass

    def __len__(self):
        return len(self.data)
        pass

    def __getitem__(self, index):
        if self.train:
            label=self.data["score"][index]
            cat=self.data.iloc[index]["title"]+". "+self.data.iloc[index]["sentence"]
            inputs = tokenizer(cat, return_tensors="pt")
            output = model_transformer(**inputs)
            embeddings=output.last_hidden_state.squeeze()[1]
            #result = nlp(cat)[0]
            return embeddings,label
        
        else:
            cat=self.data.iloc[index]["title"]+". "+self.data.iloc[index]["sentence"]
            inputs = tokenizer(cat, return_tensors="pt")
            output = model_transformer(**inputs)
            embeddings=output.last_hidden_state.squeeze()[1]
            return embeddings
        pass

In [14]:
train_dataset = ReviewDataset(train_val)
test_dataset = ReviewDataset(test_val,train=False )

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=4, pin_memory=True)
# Additional code if needed

In [16]:
print(train_dataset[1])

(tensor([-2.1767e-01,  1.9591e-01,  3.3385e-01,  1.2837e-01,  4.9409e-01,
         3.5498e-01, -1.8685e-01,  3.4219e-01,  3.9462e-02, -4.4583e-01,
        -1.0806e-01, -2.3602e-02,  1.2929e-02, -9.7192e-02, -3.7490e-01,
        -1.2347e-01,  4.6782e-01, -1.5139e-01, -1.7128e-01, -1.9161e-01,
         5.8029e-01, -1.7494e-01,  1.8317e-02,  3.1652e-01, -1.1008e-01,
         3.8947e-01,  3.1195e-01,  2.1515e-01, -2.5271e-01, -2.1922e-01,
         6.4876e-01, -2.7088e-02, -8.3678e-02, -2.0793e-01, -6.3247e-01,
        -7.7573e-01,  1.2777e-01,  1.4343e-01, -1.0237e-01,  5.4094e-01,
         7.4926e-02, -3.1621e-01,  4.1345e-01,  3.9535e-01, -1.3617e-01,
        -1.2860e-01,  8.1023e-03,  1.6361e-01,  2.0293e-02, -7.5038e-01,
        -5.1290e-02,  2.8841e-01,  4.8107e-02,  2.9240e-01, -1.0673e-01,
         5.6185e-01, -4.5827e-01,  3.3699e-01, -2.2345e-01, -2.9871e-01,
         1.3222e-01, -7.1173e-02,  1.6139e-01, -2.7798e-01,  2.5079e-01,
         1.6671e-01,  2.5110e-02, -3.2889e-01, -4.

In [17]:
# TODO: Fill out MyModule
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1=nn.Linear(768, 2048)
        self.l2=nn.Linear(2048,1)
        #self.l3=nn.Linear(73,1)
    def forward(self, x):
        x=self.l1(x)
        x=10*F.softmax(x)
        x=self.l2(x)
        #x=nn.ReLU(x)
        #x=nn.Flatten(x)
        #x=self.l3(x)
        #x=10*F.softmax(x)
        return x


model = MyModule().to(DEVICE)

In [ ]:
# TODO: Setup loss function, optimiser, and scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = nn.MSELoss()
#optim = torch.optim.SGD(model.parameters(), lr=0.001,momentum=0.9)
scheduler = None

model.train()
for epoch in range(NUM_EPOCHS):
    print(f'Current epoche:{epoch}')
    run_loss = 0
    model.train()
    for [X,y] in tqdm(train_loader,total=len(train_loader)):
        #batch = {k: v.to(device) for k, v in batch.items()}
        X=X.to(DEVICE)
        y=y.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(X)
        #loss = outputs.loss
        loss=criterion(outputs,y)
        loss.backward()
        optimizer.step()
        #scheduler.step()
        run_loss = run_loss + loss.item()
        if i % 200 == 199:    # print every 1000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {run_loss / 1000:.3f}')
            run_loss=0
        # TODO: Set up training loop

In [ ]:
model.eval()
with torch.no_grad():
    results = []
    for batch in tqdm(test_loader, total=len(test_loader)):
        batch = batch.to(DEVICE)
        predicted = model(batch)
        # TODO: Set up evaluation loop
        results.append(predicted)
    predictions = np.vstack(results)
np.savetxt("results.txt", predictions, fmt='%i')
print("Results saved to results.txt")
    #with open("result.txt", "w") as f:
        #for val in np.concatenate(results):
            #f.write(f"{val}\n")